In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from PIL import Image
from scipy import sparse
import cv2
from pymatreader import read_mat
# from extract_graph import dic_to_sparse
from util import get_path, shift_skeleton
from plotutil import show_im,overlap, show_im_rgb, plot_nodes, plot_nodes_from_list,plot_t_tp1
from extract_graph import generate_graph_tab_from_skeleton,generate_nx_graph_from_skeleton,generate_skeleton,clean
import networkx as nx
from node_id import second_identification, whole_movement_identification,first_identification,relabel_nodes, clean_nodes, orient
from extract_graph import dic_to_sparse, from_sparse_to_graph, generate_nx_graph, prune_graph, from_nx_to_tab, from_nx_to_tab_matlab,sparse_to_doc, connections_pixel_list_to_tab, transform_list
from sparse_util import dilate, zhangSuen
from realign import realign, reconnect, realign2
from util import get_path
import pandas as pd
from datetime import datetime,timedelta
import ast
import imageio


In [2]:
plate = 13
date_init = datetime(2020,7,1,19,57)
dates_datetime = [date_init+timedelta(hours=4)*i for i in range(24)]
dates = [f'{0 if date.month<10 else ""}{date.month}{0 if date.day<10 else ""}{date.day}_{0 if date.hour<10 else ""}{date.hour}{0 if date.minute<10 else ""}{date.minute}' for date in dates_datetime]

In [3]:
params=read_mat('//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE/Plate13_20200627/InitialParameters.mat')

In [4]:
# 20104, 21052
xs=5384
ys=15468

In [5]:
x1=params['x1']
x2=params['x2']
y0=params['y0']
Sf=params['Sf']
L0center=params['L0center']

In [6]:
date = dates[0]

In [7]:
date_plate = f'/2020{date}'
plate_str = f'_Plate{plate}'
findingbaits = read_mat('//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE'+date_plate+plate_str+'/Analysis/FindingBaits.mat')

In [8]:
Rcenter=findingbaits['Rcenter']
Rradii = findingbaits['Rradii']
Lcenter = findingbaits['Lcenter']
t = findingbaits['t']
Lradii = findingbaits['Lradii']

In [9]:
Rcenter,Lcenter

(array([38555., 15482.]), array([10197., 15351.]))

In [10]:
path_tile='//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE'+date_plate+plate_str+'/Img/TileConfiguration.registered.txt'

In [11]:
tileconfig = pd.read_table(path_tile,sep=';',skiprows=4,header=None,converters={2 : ast.literal_eval},skipinitialspace=True)

In [12]:
xs_yss=list(tileconfig[2])

In [13]:
xes= [xs_ys[0] for xs_ys in xs_yss]
yes= [xs_ys[1] for xs_ys in xs_yss]

In [14]:
cmin=np.min(xes)
cmax=np.max(xes)
rmin = np.min(yes)
rmax = np.max(yes)

In [15]:
S0     = [rmax+3000+abs(rmin),cmax+4096+abs(cmin)]
theta  = np.arctan((Rcenter[1]-Lcenter[1])/(Rcenter[0]-Lcenter[0]))
Sry    =  S0[0]*np.cos(abs(theta))+S0[1]*np.sin(abs(theta))

In [16]:
C = np.tan(-np.arctan((ys + y0 - 1 - L0center[1] - np.sqrt(Lcenter[0]**2+(S0[0]-Lcenter[1])**2)*np.sin(np.arctan((S0[0]-Lcenter[1])/Lcenter[0])+theta))/(xs + (x2+x1)/2 - (Sf[0]-y0) - 1 - L0center[0] + np.sqrt(Lcenter[0]**2+(S0[0]-Lcenter[1])**2)*np.cos(np.arctan((S0[0]-Lcenter[1])/Lcenter[0])+theta))) - theta)
ximg = - (ys + y0 - 1 - L0center[1] - np.sqrt(Lcenter[0]**2+(S0[0]-Lcenter[1])**2)*np.sin(np.arctan((S0[0]-Lcenter[1])/Lcenter[0])+theta))/(np.sqrt(1+C**2)*np.sin(np.arctan(C)+theta)) 
yimg = S0[0] - C*ximg;

In [17]:
def find(xsub,ysub,x,y):
    indexes=[]
    for i in range(len(xsub)):
        if x>=xsub[i]-cmin and x<xsub[i]-cmin+4096 and y>=ysub[i]-rmin and y < ysub[i]-rmin+3000:
            indexes.append(i)
    return(indexes)

In [18]:
ximg

7062.9490943354385

In [19]:
indsImg = find(xes,yes,ximg,yimg)
possImg  = [ximg-np.array(xes)[indsImg]+cmin+1, yimg-np.array(yes)[indsImg]+rmin+1];

In [20]:
possImg

[array([3865.23344551,  609.98094366]), array([1439.33955665, 1376.09704688])]

In [21]:
paths=[]
for index in indsImg:
    paths.append('//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE'+date_plate+plate_str+'/Img/'+tileconfig[0][index])

In [26]:
i=1

In [27]:
im = imageio.imread(paths[i])


In [28]:
plot_t_tp1([0],[],{0:(possImg[1][i],possImg[0][i])},None,im,im)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [165]:
path='//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE/Plate13_20200627/InitialParameters.mat'
def find_image_pos_13(xs,ys,t,path_param=path):
    params=read_mat(path_param)
    x1=params['x1']
    x2=params['x2']
    y0=params['y0']
    Sf=params['Sf']
    L0center=params['L0center']
    date = dates[t]

In [5]:
from scipy import sparse
S=sparse.dok_matrix((20800, 46000))

In [6]:
for i in range(16000):
    S[20,20]=1